In [ ]:
# If you don't have the moduel please install them for this notebook
#!pip install Bio

In [ ]:
import re
import numpy as np
import pandas as pd
from collections import Counter
import itertools
from Bio.SeqUtils.ProtParam import ProteinAnalysis

from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score, roc_auc_score,
    confusion_matrix, classification_report
)

import matplotlib.pyplot as plt
import seaborn as sns

# Read in data that was generated from the analysis and preprocessing steps

In [ ]:
# Read in data
data = pd.read_csv('metadata_org_w_features.csv')

# Information and sample data

In [ ]:
# information on data
data.info()

In [ ]:
# describe on data
data.describe()

In [ ]:
data.head()

# Extract the relevant fields from the dataset

In [ ]:
# Entry and ProteinClass
df = data.loc[:,['Entry', 'ProteinClass']]
# PDB
selected_PDB = data.loc[:,['Selected_PDB']]
# Sequence
seq = data.loc[:,['CleanSequence']]
# Sequence Length
seq_L = data.loc[:,['SequenceLength']]
# Amino Acid Frequencies
freq = data.iloc[:, 5 : 5+20]
# All possible dipeptide frequencies
dipep = data.iloc[:, 5+20 : 5+20 + 400]
# Reduced Amino Acid Alphabet Frequencies
red_freq = data.iloc[:, 5+20 + 400 : 5+20 + 400 + 5]
# N-Gram Profiles of Reduced Amino Acid Alphabet
red_ngram = data.iloc[:, 5+20 + 400 + 5 : 5+20 + 400 + 5 + 150]
# Protein Properties
prop = data.iloc[:, 5+20 + 400 + 5 + 150 :]

In [ ]:
prop.head()

# You can use this section if you wanted to add more fetures to the data

In [ ]:
## To Do ## 
# Add one new feature to this data, you don't need to use it for the model

# Construct Dataset with any of the extracted relevant fields

In [ ]:
# This is an example of construction of the data with freq and red_freq, but you can add and make any field
Dataset = pd.concat([df, freq, red_freq, prop, red_ngram], axis=1)
Dataset.head()

In [ ]:
## To Do ##
# Construct your own dataset

# Processing Steps

In [ ]:
## To Do ##

def processing_data(df):
    # Define X and y
    y = df['ProteinClass']
    X = df.drop(['Entry', 'ProteinClass'], axis=1)
    if 0:
        print("Feature sample:")
        print(X.head())
        print("\nTarget sample:")
        print(y.head())

    # Split the data
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42)

    # Feature scaling (important for logistic regression, gradient boosting, MLP, etc.)
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    ## To Do ##
    # Add a max_min normalization for the X_train and X_test below


    # Defined labels
    labels = np.array(['ATPase', 'Aquaporin', 'Channel', 'GPCR', 'Integrin', 'MHC',
       'Phosphatase', 'Protease', 'RTK', 'Ser:Thr'])
    # Create a Label Encoder
    label_encoder = LabelEncoder()
    # Fit the label encoder on all possible labels, will make the labels [0, 1, 2, 3 ...]
    label_encoder.fit(labels)
    # Transform both training and testing labels
    y_train_label_encoded = label_encoder.transform(y_train)
    y_test_label_encoded = label_encoder.transform(y_test)
    print('first sample label encoded:', y_train_label_encoded[0])

    ## To Do ##
    # Add a one hot encoder to the labels
    # Create a One Hot Encoder

    # Fit the one hot encoder on all possible labels, will make the labels 0 -> [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]
 
    # Transform both training and testing labels like the above code
  
    
    

    ### This is where you can start to add more normalization or processing approches if you would like ###

    return label_encoder, X_train, X_train_scaled, X_test, X_test_scaled, y_train, y_train_label_encoded, y_test, y_test_label_encoded
    

In [ ]:
label_encoder, X_train, X_train_scaled, X_test, X_test_scaled, y_train, y_train_encoded, y_test, y_test_encoded  = processing_data(Dataset)

# Creating Model

In [ ]:
# random model
class random_model:
    def __init__(self):
        pass
    def fit(self,X, y):
        pass
    def predict(self,X):
        pred = []
        for x in X:
            pred.append(np.random.randint(10))
        return np.array(pred)
    
# Initialize my random model
my_model = random_model()
# Initialize Logistic Regression model
lr_model = LogisticRegression( C = 1)

## To Do ##
# Create one model here like random forest, leanear model, etc and initialize it
# you can use sklear, torch, etc 

# Train Model

In [ ]:
# Fit My Model to the training data
my_model.fit(X_train, y_train)

# Fit the model to the training data
lr_model.fit(X_train_scaled, y_train)

## To Do ##
# Fit your model to the training data you can use
# raw, scaled, or other forms

# Evaluating model

In [ ]:
def evaluate_model(model, X_test, y_test, model_name):

    y_pred = model.predict(X_test)

    # Evaluation
    accuracy = accuracy_score(y_test, y_pred)
    print(f"\n=== {model_name} ===")
    print(f"Accuracy for testing data: {accuracy:.4f}")
    return y_pred

In [ ]:
# Evaluate my model
y_pred_my_model = evaluate_model(my_model, X_test_scaled, y_test_encoded, model_name="My Random Model")
# Convert back to labels
y_pred_my_model = label_encoder.inverse_transform(y_pred_my_model)
# Print my model prediction
# print(y_pred_my_model)

# Evaluate Logistic Regression
y_pred_lr_model = evaluate_model(lr_model, X_test_scaled, y_test, model_name="Logistic Regression")
# Print lr model prediction
# print(y_pred_my_model)


## To Do ##
# Evaluate your model


# Save Format

In [ ]:
def save_predictions(_fn, _y_pred, _df):
    import csv
    with open(_fn, 'w') as fout:
        writer = csv.writer(fout, delimiter=',', lineterminator='\n')
        writer.writerow(['Entry', 'ProteinClass'])
        for y, Entry in zip(_df['Entry'], _y_pred):
            writer.writerow([y, Entry])